In [1]:
import pandas as pd
import numpy as np
from cross_validation import *
from gradient_boost import predict_xgboost
from read_write import read_merged_data, load_pickled_model
from plotting_functions import scatter_plot_eda
from helper import add_rolling_averages
import matplotlib.pyplot as plt

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Functions
def get_finite_residual(df):
    return df[pd.notnull(df.Residual)]

def add_exponential_smoothing(df, dictionary):
    for key, value in dictionary.iteritems():
        for window in value:
            # Name of new feature
            name = 'Last' + str(window) + 'ExponentialSmoothingAverage' + key

            # Drop column to prevent duplicates
            if name in df.columns:
                df.pop(name)

            # Get exponential smoothing rolling average
            df[name] = pd.ewma(df[key], halflife = window).shift(1)

def add_residuals(df, element):
    """
    Input:
        df -- DataFrame
        element -- string of element we are getting residuals for
    Output:
        DataFrame with residuals column added
    """
    # Load model
    m = load_pickled_model('{}GradientBoostedRegressor'.format(element))
    
    # Predict
    predictions, filtered_df = predict_xgboost(df, 
                                               element = element, 
                                               data_info = data_info)

    prediction_df = pd.DataFrame()
    prediction_df['Predicted{}'.format(element)] = predictions
    prediction_df['Player'] = filtered_df.Player
    prediction_df['Date'] = filtered_df.Date
    prediction_df['Team'] = filtered_df.Team
    return df.merge(prediction_df, on = ['Player', 'Date', 'Team'], how = 'left')

In [3]:
# Load data
df = pd.read_csv('data/merged_data.csv')

In [4]:
# Filtering info
data_info = cv_method(method = k_folds_cv,
                      splits = 5,
                      start_date = '1999-01-01',
                      end_date = '2016-09-01',
                      minutes_cutoff = 3)

In [ ]:
# # Examine data
"""
1. There is a duplicate season column
"""
print "Num rows: {}".format(len(df))
for column in df.columns:
    print column

In [5]:
# Add residuals
df = add_residuals(df, 'FanDuelScore')

In [ ]:
# Explore features to predict residuals
scatter_plot_eda(df, 'PlayerPTS', 'PredictedFanDuelScore', 'cat')

In [ ]:
df[['PredictedFanDuelScore_y','FanDuelScore', 'Player']]

In [ ]:
filtered_df = align_players(filtered_df)
print len(filtered_df)

# Display DataFrame
filtered_df[pd.notnull(filtered_df.StarterLineupOrder)].loc[:,['Player_x','StarterLineupOrder', 'Residual_x', 'BucketedMinutes_x', 'Player_y', 'BucketedMinutes_y', 'Residual_y']]

# Remove all rows with NaN for 'Residual_y'
filtered_df.dropna(subset = ['PlayerDefenseMatchupResidual'], inplace = True, axis = 0)
print len(filtered_df)

In [ ]:
# Experimentation shows that 30.85 is the optimal half-life decay parameter for predicting 'Residual_y'
# based on the exponential smoothing of 

player_exponential_smoothing_dict = {'PlayerDefenseMatchupResidual': [30.80,30.85,30.90]}
def add_position_defense_stats(df):
    # Remove all rows with NaN for 'Residual_y'
    filtered_df = df.dropna(subset = ['PlayerDefenseMatchupResidual'], axis = 0)
    filtered_df = filtered_df.groupby(['Player']).apply(GB_apply_player_position)
    return filtered_df
filtered_df = add_position_defense_stats(filtered_df)

In [ ]:
for key, value in player_exponential_smoothing_dict.iteritems():
    for window in value:
        scatter_plot_eda(filtered_df, key + 'ExponentialSmoothing' + str(window), 'PlayerDefenseMatchupResidual', 'cat')

In [ ]:
f = filtered_df.groupby(['Date', 'Team', 'Frontcourt']).aggregate(np.sum)
f.reset_index(level=['Team', 'Date', 'Frontcourt'], inplace=True)
for key, value in player_exponential_smoothing_dict.iteritems():
    for window in value:
        old_name = key + 'ExponentialSmoothing' + str(window)
        rename_dict = {old_name : 'PositionGroup' + old_name}
        f.rename(columns = rename_dict, inplace = True)

In [ ]:
features = ['PositionGroup'+ key + 'ExponentialSmoothing' + str(window) for window in value for key, value in player_exponential_smoothing_dict.iteritems()]
features += ['Date', 'Team', 'Frontcourt']
filtered_df = filtered_df.merge(f[features], on = ['Date', 'Team', 'Frontcourt'], how = 'left')

In [ ]:
for key, value in player_exponential_smoothing_dict.iteritems():
    for window in value:
        scatter_plot_eda(filtered_df, 'PositionGroup' + key + 'ExponentialSmoothing' + str(window), 'PlayerDefenseMatchupResidual', 'cat')

In [ ]:
features = [key + 'ExponentialSmoothing' + str(window) for window in value for key, value in player_exponential_smoothing_dict.iteritems()]
features += ['PlayerDefenseMatchup', 'Date', 'Team']
df = df.merge(filtered_df[features], left_on = ['Player', 'Date', 'Opp'], right_on = ['PlayerDefenseMatchup', 'Date', 'Team'], how = 'left')